# Домашнее задание ШАД МТС по теме Оптимизация 1

## Задание

Решить задачу линейного программирования симплекс-методом (с использованием симплекс таблиц). Итоговый результат проверить через симплекс-метод из scipy.optimize (см. документацию).

$L = 2x_1 + 3x_2 - x_4 \to max$

$\begin{cases}
    2x_1 - x_2 - 2x_4 + x_5 = 16 \\
    3x_1 + 2x_2 + x_3 - 3x_4 = 18 \\
    -x_1 + 3x_2 + 4x_4 + x_6 = 24 \\
    x_1, \dotso, x_6 \ge 0
\end{cases}$

## Ручное решение симплекс-методом

Сводим задачу к минимизации

$L = -2x_1 - 3x_2 + x_4 \to min$

А переменные $x_3$, $x_5$, $x_6$ считаем базисными, т.к. каждая из них входит только в одно уравнение, причем с единичным коэффициентом

| Базис | $x_1$ | $x_2$ | $x_3$ | $x_4$ | $x_5$ | $x_6$ | b |
|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:---:|
| $x_5$ | 2 | -1 | 0 | -2 | 1 | 0 | 16 |
| $x_3$ | 3 | 2 | 1 | -3 | 0 | 0 | 18 |
| $x_6$ | -1 | **3** | 0 | 4 | 0 | 1 | 24 |
| $z$ | -2 | -3* | 0 | 1 | 0 | 0 | 0 |

Начальным опорным планом является точка $x_0 = (0, 0, 18, 0, 16, 24)$.

Поскольку в $z$-строке имеются отрицательные элементы, минимум целевой функции в точке $x_0$ не достигается.

Выбираем разрешиющим столбцом $x_2$, т.к. он содержит минимальный элемент. Тогда разрешающим элементом будет элемент 3 в строке $x_6$, т.к.

$min ( \large \frac{18}{2} \normalsize; \large \frac{24}{3} \normalsize) = \large \frac{24}{3} \normalsize = 8$

Выполняем замену базиса $x_6$ на $x_2$

| Базис | $x_1$ | $x_2$ | $x_3$ | $x_4$ | $x_5$ | $x_6$ | b |
|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:---:|
| $x_5$ | 5/3 | 0 | 0 | -2/3 | 1 | 1/3 | 24 |
| $x_3$ | **11/3** | 0 | 1 | -17/3 | 0 | -2/3 | 2 |
| $x_2$ | -1/3 | 1 | 0 | 4/3 | 0 | 1/3 | 8 |
| $z$ | -3* | 0 | 0 | 5 | 0 | 1 | 24 |

На данной итерации опорным планом является точка $x_1 = (0, 8, 2, 0, 24, 0)$.

Поскольку в $z$-строке имеются отрицательные элементы, минимум целевой функции в точке $x_1$ не достигается.

Выбираем разрешиющим столбцом $x_1$, т.к. он содержит минимальный элемент. Тогда разрешающим элементом будет элемент 11/3 в строке $x_3$, т.к.

$min ( \large \frac{24}{5/3} \normalsize; \large \frac{2}{11/3} \normalsize) = \large \frac{2}{11/3} \normalsize = \large \frac{6}{11} \normalsize$

Выполняем замену базиса $x_3$ на $x_1$

| Базис | $x_1$ | $x_2$ | $x_3$ | $x_4$ | $x_5$ | $x_6$ | b |
|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:---:|
| $x_5$ | 0 | 0 | -5/11 | 21/11 | 1 | 7/11 | 254/11 |
| $x_1$ | 1 | 0 | 3/11 | -17/11 | 0 | -2/11 | 6/11 |
| $x_2$ | 0 | 1 | 0 | 1/11 | 9/11 | 0 | 90/11 |
| $z$ | 0 | 0 | 9/11 | 4/11 | 0 | 5/11 | 282/11 |

На данной итерации опорным планом является точка $x_2 = (6/11, 90/11, 0, 0, 254/11, 0)$.

Поскольку в $z$-строке все элементы положительные, то мы достигли экстремума функции, останавливаем итерации.

Значение функции в этой точке равно последнему элементу в $z$-строке: 282/11 или вычисляя по попрному плану:

$\large \frac{282}{11} \normalsize = 2 \large \frac{6}{11} \normalsize + 3 \large \frac{90}{11} \normalsize \approx 25,64$

## Проверка корректности решения

Поскольку начиная с версии SciPy 1.11.0 method='simplex' удален, то будем использовать рекомендуемый [method='highs'](https://docs.scipy.org/doc/scipy/reference/optimize.linprog-highs.html#optimize-linprog-highs)

In [23]:
from scipy.optimize import linprog
import numpy as np

c = np.array([-2, -3, 0, 1, 0, 0])
A = np.array([
    [2, -1, 0, -2, 1, 0],
    [3, 2, 1, -3, 0, 0],
    [-1, 3, 0, 4, 0, 1]
])
b = np.array([16, 18, 24])

res = linprog(c, A_eq=A, b_eq=b, method='highs')
print(f'Оптимальное значение функции: {-res.fun}')
print(f'Опорный план: {res.x}')


Оптимальное значение функции: 25.636363636363637
Оптимальный опорный план: [ 0.54545455  8.18181818  0.          0.         23.09090909  0.        ]
